In [1]:
import numpy as np
import pandas as pd
import os
import git
from git import RemoteProgress
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re

In [2]:
remote_link = "https://github.com/spring-projects/spring-social"
local_link = "spring-social"

repo = Repo(local_link)
fixing_commit = "5151e1158e1ed51369e7aba9e8092930d28c31d3"
affected_file = "spring-social-core/src/main/java/org/springframework/social/support/ClientHttpRequestFactorySelector.java"
commit = repo.commit(fixing_commit)

In [3]:
def check_latest_commit(start, end):
    commits = {}
    blames = repo.git.blame("-L"+str(start)+","+str(end), commit.parents[0], affected_file).splitlines()
    latest_commit = None
    latest_commit_time_obj = None
    
    for blame in blames:
        words = blame.split()
        commit_id = words[0]
        
        d = list(filter(re.compile("\d*-\d*-\d*").match, words))
        t = list(filter(re.compile("\d*:\d*:\d*").match, words))
        
        time = d[0] +" "+ t[0]
        
        #print (time)
        time_obj = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')

        if(latest_commit_time_obj == None):
            latest_commit_time_obj = time_obj
            latest_commit = commit_id
            continue
            
        if(latest_commit_time_obj<time_obj):
            latest_commit_time_obj = time_obj
            latest_commit = commit_id
        
    return latest_commit

In [4]:
# Task 3

deleted_lines = [21, 22, 23, 24, 34, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 132, 133, 134, 135, 136, 137, 138, 139, 141]

encloing_scope_for_added_lines_and_number_of_lines = [[1, 160, 1], [1, 160, 1], [1, 160, 1], [1, 160, 1],[1, 160, 1],[1, 160, 1],[1, 160, 1],[1, 160, 1],[1, 160, 1],[1, 160, 1], [122, 146, 2], [122, 146, 18]]

commits = {}

# count delete
for line in deleted_lines:
    blame = repo.git.blame("-L"+str(line)+","+str(line), commit.parents[0], affected_file).splitlines()
    commit_id = blame[0].split()[0]
    if commit_id in commits:
        commits[commit_id] = commits[commit_id] + 1
    else:
        commits[commit_id] = 1
        
# count add
for start, end, n in encloing_scope_for_added_lines_and_number_of_lines:
    commit_id = check_latest_commit(start, end)
    if commit_id in commits:
        commits[commit_id] = commits[commit_id] + (n)
    else:
        commits[commit_id] = 1
        
print (commits)
counter = 0
VCC = None
for i in commits:
    if(commits[i]>counter):
        counter = commits[i]
        VCC = i
    
print ("VCC is", VCC)

{'d3a8adac8': 21, '8e3f46ba7': 24, '2199cf84d': 10}
VCC is 8e3f46ba7


In [5]:
# Task 5
commit_VCC = repo.commit(VCC)
show = repo.git.show("--shortstat", VCC, affected_file).splitlines()

title = show[4]

if(len(show)==7):
    message = None
else:
    for i in range(len(show)-7):
        message += show[5+i]

print ("Title:", title)
if message == None:
    print ("Message:", "None")
else:
    print ("Message:", message)

Title:     Polish and add JavaDoc to all-trust pull request.
Message: None


In [6]:
summary = show[len(show)-1]
numbers = re.findall(r'\d+', summary)
number_of_files_affected = numbers[0]
print (number_of_files_affected, "affected files")
#print(commit_VCC.stats.files)

1 affected files


In [7]:
directories = []

for file in commit_VCC.stats.files:
    directories.append(file[:file.rfind('/')])
    
directories = set(directories)
number_of_directories_affected = len(directories)
print(number_of_directories_affected, "affected directories")

1 affected directories


In [8]:
print (numbers[2], "lines of code were deleted including comments and blank lines")
print (numbers[1], "lines of code were added including comments and blank lines")

7 lines of code were deleted including comments and blank lines
12 lines of code were added including comments and blank lines


In [35]:
diff = repo.git.diff("-w", VCC + "^", VCC, affected_file).splitlines()

add_counter = 0
delete_counter = 0

within_comment = False
for line in diff:
    
    if(line[:3] == '+++' or line[:3] == "---"):
        continue
        
    if(within_comment):
        if(line.find('*/')!=-1):
            within_comment = False
        continue
        
    if(line[0]=="+" or line[0]=="-"):
        if(line.find('/*')!=-1):
            within_comment = True
            continue
        if(line[0]=="+"):
            add_counter += 1
        if(line[0]=="-"):
            delete_counter += 1
            
print(delete_counter, "lines of code were deleted excluding comments and blank lines")
print(add_counter, "lines of code were added excluding comments and blank lines")

0 lines of code were deleted excluding comments and blank lines
0 lines of code were added excluding comments and blank lines


In [70]:
words = show[2].split()

VCC_time = words[5]+" "+words[2]+" "+words[3]+" "+words[4]
VCC_time_obj = datetime.datetime.strptime(VCC_time, '%Y %b %d %H:%M:%S')

for file in commit_VCC.stats.files:
    log = repo.git.log(file).splitlines()
    w = log[2].split()
    file_time = w[5]+" "+w[2]+" "+w[3]+" "+w[4]
    file_time_obj = datetime.datetime.strptime(file_time, '%Y %b %d %H:%M:%S')
    
    log_data = repo.git.log("--oneline", "--follow",file).splitlines()
    log_name_data = repo.git.log("--pretty=format:%an", "--follow", file).splitlines()
    del log_name_data[-1]
    
    log_name_data = {i:log_name_data.count(i) for i in log_name_data}
    
    print (file,":")
    print (file_time_obj - VCC_time_obj, "were between the current VCC and the previous commit")
    print ("been modified", len(log_data)-1, "times in the past since creation")
    print (log_name_data, "have modified this file since its creation")
    
    print ()

spring-social-core/src/main/java/org/springframework/social/support/ClientHttpRequestFactorySelector.java :
316 days, 1:00:55 were between the current VCC and the previous commit
been modified 19 times in the past since creation
{'Craig Walls': 10, 'Gim Gyung Jin': 2, 'Keith Donald': 5, 'Roy Clarkson': 2} have modified this file since its creation

